## Loading Label Data

In [ ]:
from scipy.io import loadmat

In [ ]:
bmw = loadmat('bmw10_release/bmw10_annos.mat')
bmw

In [ ]:
bmw['annos'][0][0][1][0][0]

In [ ]:
cars = loadmat('devkit/cars_meta.mat')
cars

In [ ]:
cars['class_names'][0][31]

In [ ]:
loadmat('devkit/cars_train_annos.mat')['annotations'][0][256]

## Keras Exploration

In [ ]:
import keras

In [ ]:
keras.applications.vgg19.VGG19(include_top=False, weights='imagenet', input_tensor=None)

### Preprocessing Images

In [ ]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [ ]:
#Create photo transform generator

datagen = ImageDataGenerator(
    rotation_range = 40,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    rescale = 1./255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    fill_mode = 'nearest'
)

### Testing Transformation

In [ ]:
img = load_img('data/train/bmw/149105744.jpg')
x = img_to_array(img)
x = x.reshape((1,) + x.shape)

i = 0
for batch in datagen.flow(x, batch_size = 1,
                         save_to_dir = 'preview', save_prefix = 'bmw', save_format = 'jpeg'):
    i += 1
    if i > 20:
        break

## Transformation Testing for BMW models

In [ ]:
datagen = ImageDataGenerator(
    horizontal_flip = True,
    rescale = 1./255,
    rotation_range = 40,
    fill_mode = 'nearest'
)

model = Sequential()
model.add(ZeroPadding2D((1, 1), input_shape=(3, 224, 224)))


# img = load_img('front_angle/transform_tests/BMW_2_FrontAngle_003.jpg')
# x = img_to_array(img)
# x = x.reshape((1,) + x.shape)

# i = 0
# for batch in 
generator = datagen.flow_from_directory('front_angle/transform_tests',
                            target_size = (244,244),
                            batch_size = 1,
                         save_to_dir = 'front_angle/transform_tests', 
                                         save_prefix = 'bmw2', 
                                         save_format = 'jpeg')

bottleneck_features_train = model.predict_generator(generator, 6)
np.save('bottleneck_features_train.npy', bottleneck_features_train)

#     i += 1
#     if i > 4:
#         break

## Trying a Re-Train on VGG-19

In [ ]:
import os
import h5py
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.layers import Activation, Dropout, Flatten, Dense, Input
from keras import backend as K
from keras import optimizers

### Subset Test

In [ ]:
# path to the model weights file.
weights_path = 'weights/vgg19_weights.h5'
top_model_weights_path = 'bottleneck_fc_model.h5'
# dimensions of our images.
img_width, img_height = 224, 224

train_data_dir = 'data_sub/train'
validation_data_dir = 'data_sub/validation'
nb_train_samples = 200
nb_validation_samples = 150
nb_epoch = 50

In [ ]:
def save_bottlebeck_features():
    datagen = ImageDataGenerator(rescale=1., featurewise_center=True) #(rescale=1./255)
    datagen.mean=np.array([103.939, 116.779, 123.68],dtype=np.float32).reshape(3,1,1)

    # build the VGG19 network
    model = Sequential()
    model.add(ZeroPadding2D((1, 1), input_shape=(3, img_width, img_height)))
    
    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))
    
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))
    
    # load the weights of the VGG19 networks
    assert os.path.exists(weights_path), 'Model weights not found (see "weights_path" variable in script).'
    f = h5py.File(weights_path)
    for k in range(f.attrs['nb_layers']):
        if k >= len(model.layers):
            # we don't look at the last (fully-connected) layers in the savefile
            break
        g = f['layer_{}'.format(k)]
        weights = [g['param_{}'.format(p)] for p in range(g.attrs['nb_params'])]
        model.layers[k].set_weights(weights)
    f.close()
    print('Model loaded.')

    generator = datagen.flow_from_directory(
            train_data_dir,
            target_size=(img_width, img_height),
            batch_size=32,
            class_mode=None,
            shuffle=False)
    
    bottleneck_features_train = model.predict_generator(generator, nb_train_samples)
    np.save('bottleneck_features_train.npy', bottleneck_features_train)
    
    print('done with train.')

    generator = datagen.flow_from_directory(
            validation_data_dir,
            target_size=(img_width, img_height),
            batch_size=32,
            class_mode=None,
            shuffle=False)
    
    bottleneck_features_validation = model.predict_generator(generator, nb_validation_samples)
    np.save('bottleneck_features_validation.npy', bottleneck_features_validation)
    
    print('done with validation.')

In [ ]:
save_bottlebeck_features()

In [ ]:
def train_top_model():
    train_data = np.load('bottleneck_features_train.npy')
    train_labels = np.array([1] * 100 + [0] * 100)

    validation_data = np.load('bottleneck_features_validation.npy')
    validation_labels = np.array([1] * 50 + [0] * 100)

    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer=optimizers.RMSprop(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])
    
    model.fit(train_data, train_labels,
              nb_epoch=nb_epoch, batch_size=32,
              validation_data=(validation_data, validation_labels))
    model.save_weights(top_model_weights_path)

In [ ]:
train_top_model()

224x224 = 0.5667

### WIth Fine Tuning

In [ ]:
# path to the model weights files.
weights_path = 'weights/vgg19_weights.h5'
top_model_weights_path = 'bottleneck_fc_model.h5'
# dimensions of our images.
img_width, img_height = 224, 224

train_data_dir = 'data_sub/train'
validation_data_dir = 'data_sub/validation'
nb_train_samples = 200
nb_validation_samples = 150
nb_epoch = 50

# build the VGG16 network
model = Sequential()
model.add(ZeroPadding2D((1, 1), input_shape=(3, img_width, img_height)))

model.add(Convolution2D(64, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(64, 3, 3, activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, 3, 3, activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, 3, 3, activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, 3, 3, activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, 3, 3, activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

# load the weights of the VGG16 networks
# (trained on ImageNet, won the ILSVRC competition in 2014)
# note: when there is a complete match between your model definition
# and your weight savefile, you can simply call model.load_weights(filename)
assert os.path.exists(weights_path), 'Model weights not found (see "weights_path" variable in script).'
f = h5py.File(weights_path)
for k in range(f.attrs['nb_layers']):
    if k >= len(model.layers):
        # we don't look at the last (fully-connected) layers in the savefile
        break
    g = f['layer_{}'.format(k)]
    weights = [g['param_{}'.format(p)] for p in range(g.attrs['nb_params'])]
    model.layers[k].set_weights(weights)
f.close()
print('Model loaded.')

# build a classifier model to put on top of the convolutional model
top_model = Sequential()
top_model.add(Flatten(input_shape=model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1, activation='sigmoid'))

# note that it is necessary to start with a fully-trained
# classifier, including the top classifier,
# in order to successfully do fine-tuning
top_model.load_weights(top_model_weights_path)

# add the model on top of the convolutional base
model.add(top_model)

# set the first 25 layers (up to the last conv block)
# to non-trainable (weights will not be updated)
for layer in model.layers[:25]:
    layer.trainable = False

# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
        rescale=1.,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
train_datagen.mean=np.array([103.939, 116.779, 123.68],dtype=np.float32).reshape(3,1,1)

test_datagen = ImageDataGenerator(rescale=1., featurewise_center=True) #(rescale=1./255)
test_datagen.mean=np.array([103.939, 116.779, 123.68],dtype=np.float32).reshape(3,1,1)
    
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_height, img_width),
        batch_size=32,
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_height, img_width),
        batch_size=32,
        class_mode='binary')

# fine-tune the model
model.fit_generator(
        train_generator,
        samples_per_epoch=nb_train_samples,
        nb_epoch=nb_epoch,
        validation_data=validation_generator,
        nb_val_samples=nb_validation_samples)

### Full Data Set

In [ ]:
import os
import h5py
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.layers import Activation, Dropout, Flatten, Dense, Input
from keras import backend as K
from keras import optimizers

# path to the model weights file.
weights_path = 'vgg19_weights.h5'
top_model_weights_path = 'bottleneck_fc_model.h5'
# dimensions of our images.
img_width, img_height = 224, 224

train_data_dir = 'data/train'
validation_data_dir = 'data/validation'
nb_train_samples = 8525
nb_validation_samples = 1040
nb_epoch = 100

def save_bottlebeck_features():
    datagen = ImageDataGenerator(rescale=1., featurewise_center=True) #(rescale=1./255)
    datagen.mean=np.array([103.939, 116.779, 123.68],dtype=np.float32).reshape(3,1,1)

    # build the VGG19 network
    model = Sequential()
    model.add(ZeroPadding2D((1, 1), input_shape=(3, img_width, img_height)))

    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    # load the weights of the VGG19 networks
    assert os.path.exists(weights_path), 'Model weights not found (see "weights_path" variable in script).'
    f = h5py.File(weights_path)
    for k in range(f.attrs['nb_layers']):
        if k >= len(model.layers):
            # we don't look at the last (fully-connected) layers in the savefile
            break
        g = f['layer_{}'.format(k)]
        weights = [g['param_{}'.format(p)] for p in range(g.attrs['nb_params'])]
        model.layers[k].set_weights(weights)
    f.close()
    print('Model loaded.')

    generator = datagen.flow_from_directory(
            train_data_dir,
            target_size=(img_width, img_height),
            batch_size=32,
            class_mode=None,
            shuffle=False)

    bottleneck_features_train = model.predict_generator(generator, nb_train_samples)
    np.save('bottleneck_features_train.npy', bottleneck_features_train)

    print('done with train.')

    generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=32,
        class_mode=None,
        shuffle=False)

    bottleneck_features_validation = model.predict_generator(generator, nb_validation_samples)
    np.save('bottleneck_features_validation.npy', bottleneck_features_validation)

    print('done with validation.')

In [ ]:
def train_top_model():
    train_data = np.load('bottleneck_features_train.npy')
    train_labels = np.array([1] * 1009 + [0] * 7516)

    validation_data = np.load('bottleneck_features_validation.npy')
    validation_labels = np.array([1] * 520 + [0] * 520)

    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu',  init='lecun_uniform'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer=optimizers.RMSprop(lr=0.00001), loss='binary_crossentropy', metrics=['accuracy'])

    model.fit(train_data, train_labels,
              nb_epoch=nb_epoch, batch_size=32,
              validation_data=(validation_data, validation_labels))
    model.save_weights(top_model_weights_path)

In [ ]:
save_bottlebeck_features()
train_top_model()

## Trying to load model

In [1]:
from keras.models import load_model
from keras.preprocessing import image
import numpy as np
import requests
import wget

Using Theano backend.


In [ ]:
def get_image_score(image_path):
    # r = requests.get(url)
    # im = np.array(Image.open(StringIO(r.content)))
#     wget.download(image_path, out = 'app/static/car/car.jpg')

    img = image.load_img('test/car.jpg', target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)

    model = load_model('/Users/justinchien/ds/metis/Project_Kojak/app/models/convolutions.h5')
    convoluted = model.predict(x)

    model2 = load_model('/Users/justinchien/ds/metis/Project_Kojak/app/models/fronts.h5')
    prediction = model2.predict(convoluted)

    predictions = []

    for i in range(len(prediction[0])):
        predictions.append((i, prediction[0][i]))

    predictions.sort(key=lambda x: x[1], reverse = True)

    car_dict = {
        0:'aston',
        1:'audi',
        2:'bmw',
        3:'bugatti',
        4:'ferrari',
        5:'lamborghini',
        6:'mclaren'
    }

    result = (car_dict[predictions[0][0]])
    return (result)

In [ ]:
get_image_score("a")

In [ ]:
import keras

keras.__version__

In [ ]:
def preprocess_input(x):
    x /= 255.
    x -= 0.5
    x *= 2.
    return x

url = 'http://st.motortrend.com/uploads/sites/5/2013/09/Bugatti-Veyron-Grand-Sport-Vitesse-Legend-Jean-Bugatti-front-view.jpg'
wget.download(url, out = 'test/car.jpg')

img = image.load_img('test/car.jpg', target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

model = load_model('/Users/justinchien/ds/metis/Project_Kojak/app/models/convolutions.h5')
print('loaded convolutions')
convoluted = model.predict(x)

model2 = load_model('/Users/justinchien/ds/metis/Project_Kojak/app/models/fronts.h5')
print('loaded top model')
prediction = model2.predict(convoluted)

predictions = []

for i in range(len(prediction[0])):
    predictions.append((i, prediction[0][i]))

predictions.sort(key=lambda x: x[1], reverse = True)

car_dict = {
    0:'aston',
    1:'audi',
    2:'bmw',
    3:'bugatti',
    4:'ferrari',
    5:'lamborghini',
    6:'mcLaren'
}

print(car_dict[predictions[0][0]])

In [ ]:
car_dict = {
    0:'aston',
    1:'audi',
    2:'bmw',
    3:'bugatti',
    4:'ferrari',
    5:'lamborghini',
    6:'mcLaren'
}

In [ ]:
print(car_dict[predictions[0][0]], predictions[0][1]*100)
print(car_dict[predictions[1][0]], predictions[1][1]*100)

## Trying BMW angled model designation

In [ ]:
convolutions = load_model('/Users/justinchien/ds/metis/Project_Kojak/app/models/convolutions.h5')
print('loaded convolutions')

bmw_top = load_model('/Users/justinchien/ds/metis/Project_Kojak/app/models/bmws_top.h5')
print('loaded bmw top model')

In [ ]:
def preprocess_input(x):
    x /= 255.
    x -= 0.5
    x *= 2.
    return x

url = 'https://media.ed.edmunds-media.com/bmw/5-series/2016/oem/2016_bmw_5-series_sedan_535d_fq_oem_1_300.jpg'
wget.download(url, out = 'test/car.jpg')

img = image.load_img('test/car.jpg', target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

convoluted = convolutions.predict(x)

prediction = bmw_top.predict(convoluted)

predictions = []

for i in range(len(prediction[0])):
    predictions.append((i, prediction[0][i]))

predictions.sort(key=lambda x: x[1], reverse = True)

print(predictions)

os.remove('test/car.jpg')

bmw_dict = {
    0:'2 Series',
    1:'3 Series',
    2:'4 Series',
    3:'5 Series',
    4:'6 Series',
    5:'7 Series',
    6:'i3',
    7:'i8'
}

print(bmw_dict[predictions[0][0]], predictions[0][1]*100)
print(bmw_dict[predictions[1][0]], predictions[1][1]*100)

## Flipping Image Test

In [ ]:
from PIL import Image
import glob

In [ ]:
image_list = []
for filename in glob.glob('front_angle/transform_tests/*.jpeg'):
    im=Image.open(filename)
    out = im.transpose(Image.FLIP_LEFT_RIGHT)
    image_list.append(out)

In [ ]:
for i in range(len(image_list)):
    image_list[i].save('flipped{}.jpeg'.format(i))

## Trying Front or Angled Into Front Prediction

In [12]:
convolutions = load_model('/Users/justinchien/ds/metis/Project_Kojak/app/models/convolutions.h5')
print('loaded convolutions')

front_or_angle_top = load_model('/Users/justinchien/ds/metis/Project_Kojak/app/models/front_or_angle_top.h5')
print('loaded front or angle model')

fronts = load_model('/Users/justinchien/ds/metis/Project_Kojak/app/models/fronts.h5')
print('loaded fronts model')

angled = load_model('/Users/justinchien/ds/metis/Project_Kojak/app/models/angled.h5')
print('loaded angled model')

bmw_top = load_model('/Users/justinchien/ds/metis/Project_Kojak/app/models/bmws_top.h5')
print('loaded bmw top model')

/Users/justinchien/anaconda3/lib/python3.5/site-packages/keras/models.py:136: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


loaded convolutions
loaded front or angle model
loaded fronts model
loaded angled model
loaded bmw top model


In [5]:
from keras.models import load_model
from keras.preprocessing import image
import numpy as np
import requests
import wget
import os

In [17]:
def preprocess_input(x):
    x /= 255.
    x -= 0.5
    x *= 2.
    return x

url = 'http://st.automobilemag.com/uploads/sites/11/2013/11/2014-Ferrari-458-Speciale-front.jpg'
wget.download(url, out = 'test/car.jpg')

img = image.load_img('test/car.jpg', target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

convoluted = convolutions.predict(x)
prediction = front_or_angle_top.predict(convoluted)

predictions = []

for i in range(len(prediction[0])):
    predictions.append((i, prediction[0][i]))

predictions.sort(key=lambda x: x[1], reverse = True)

os.remove('test/car.jpg')

#If angled view
if predictions[0][1] > 0.5:
    print ('angled')
    #Send to Angled models for sangled
    angled_prediction = angled.predict(convoluted)
    angled_predictions = []
    
    for i in range(len(angled_prediction[0])):
        angled_predictions.append((i, angled_prediction[0][i]))
    
    angled_predictions.sort(key=lambda x:x[1], reverse = True)
    
    car_dict = {
        0:'aston',
        1:'audi',
        2:'bmw',
        3:'bugatti',
        4:'ferrari',
        5:'lamborghini',
        6:'mcLaren'
    }
    
    #if BMW, go to model designation
    if angled_predictions[0][0] == 2:
        print('BMW!')
        bmw_prediction = bmw_top.predict(convoluted)
        bmw_predictions = []

        for i in range(len(bmw_prediction[0])):
            bmw_predictions.append((i, bmw_prediction[0][i]))

        bmw_predictions.sort(key=lambda x: x[1], reverse = True)

        bmw_dict = {
            0:'2 Series',
            1:'3 Series',
            2:'4 Series',
            3:'5 Series',
            4:'6 Series',
            5:'7 Series',
            6:'i3',
            7:'i8'
        }

        print(bmw_dict[bmw_predictions[0][0]], bmw_predictions[0][1]*100)
        print(bmw_dict[bmw_predictions[1][0]], bmw_predictions[1][1]*100)
    
    #Print top two predictions for front if not BMW
    else:
        print(car_dict[angled_predictions[0][0]], angled_predictions[0][1]*100)
        print(car_dict[angled_predictions[1][0]], angled_predictions[1][1]*100) 
    
    
#If front view
else:
    print ('front')
    
    #Send to top models for front
    front_prediction = fronts.predict(convoluted)
    front_predictions = []

    for i in range(len(front_prediction[0])):
        front_predictions.append((i, front_prediction[0][i]))

    front_predictions.sort(key=lambda x: x[1], reverse = True)

    car_dict = {
        0:'aston',
        1:'audi',
        2:'bmw',
        3:'bugatti',
        4:'ferrari',
        5:'lamborghini',
        6:'mcLaren'
    }

    #Print top two predictions for front
    print(car_dict[front_predictions[0][0]], front_predictions[0][1]*100)
    print(car_dict[front_predictions[1][0]], front_predictions[1][1]*100)

front
ferrari 99.9737024307
bmw 0.0105421000626
